In [ ]:
#@title Auto-Execute Setup { display-mode: "form" }
%%javascript

// Auto-run all cells when notebook loads
setTimeout(() => {
    // Check if we're in Colab
    if (window.colab) {
        console.log('Auto-running Claude Tools processor...');
        // Run all cells
        colab.kernel.invokeFunction('notebook.RunAll', [], {});
    }
}, 3000);  // Wait 3 seconds for notebook to fully load


In [ ]:
# Auto-mount Drive without interaction
import os
from google.colab import drive
import IPython

# Attempt to auto-mount
try:
    drive.mount('/content/drive', force_remount=True)
    print('✅ Drive auto-mounted')
except:
    print('⚠️ Manual Drive authorization needed')
    drive.mount('/content/drive')

In [ ]:
# Claude Tools Auto-Processor
print('🤖 Claude Tools Auto-Processor Starting...')
print('=' * 50)

import os
import json
import time
import traceback
from datetime import datetime
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
import io

# Configuration
FOLDER_ID = '1tzHn4J3QntSLJlJNXcNJe3cLdILGEb3Z'
AUTO_RUN_DURATION = 3600  # 1 hour
POLL_INTERVAL = 2

class AutoProcessor:
    def __init__(self):
        self.folder_id = FOLDER_ID
        self.drive_service = None
        self.processed_requests = set()
        
    def initialize(self):
        """Initialize with Google Drive access"""
        try:
            # Authenticate
            auth.authenticate_user()
            self.drive_service = build('drive', 'v3')
            print(f'✅ Initialized with folder: {self.folder_id}')
            return True
        except Exception as e:
            print(f'❌ Initialization failed: {e}')
            return False
    
    def execute_code(self, code):
        """Execute code safely"""
        from io import StringIO
        import sys
        
        old_stdout = sys.stdout
        sys.stdout = StringIO()
        
        try:
            # Create isolated namespace
            namespace = {'__name__': '__main__'}
            exec(code, namespace)
            output = sys.stdout.getvalue()
            return {
                'status': 'success',
                'output': output,
                'timestamp': time.time()
            }
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e),
                'traceback': traceback.format_exc(),
                'timestamp': time.time()
            }
        finally:
            sys.stdout = old_stdout
    
    def list_requests(self):
        """Find pending requests"""
        try:
            query = f"'{self.folder_id}' in parents and name contains 'command_' and trashed=false"
            results = self.drive_service.files().list(
                q=query,
                fields="files(id, name, createdTime)",
                orderBy="createdTime"
            ).execute()
            
            requests = []
            for file in results.get('files', []):
                if file['id'] not in self.processed_requests:
                    requests.append(file)
            
            return requests
        except Exception as e:
            print(f'❌ Error listing requests: {e}')
            return []
    
    def read_request(self, file_id):
        """Read request from Drive"""
        try:
            content = self.drive_service.files().get_media(fileId=file_id).execute()
            return json.loads(content.decode('utf-8'))
        except Exception as e:
            print(f'❌ Error reading request: {e}')
            return None
    
    def write_response(self, command_id, response_data):
        """Write response to Drive"""
        try:
            response_name = f'result_{command_id}.json'
            
            # Create file
            file_metadata = {
                'name': response_name,
                'parents': [self.folder_id]
            }
            
            media = MediaIoBaseUpload(
                io.BytesIO(json.dumps(response_data, indent=2).encode('utf-8')),
                mimetype='application/json'
            )
            
            self.drive_service.files().create(
                body=file_metadata,
                media_body=media
            ).execute()
            
            print(f'✅ Response written: {response_name}')
        except Exception as e:
            print(f'❌ Error writing response: {e}')
    
    def process_request(self, request_file):
        """Process a single request"""
        try:
            # Extract command ID from filename
            command_id = request_file['name'].replace('command_', '').replace('.json', '')
            print(f'\n📋 Processing: {command_id}')
            
            # Read request
            request_data = self.read_request(request_file['id'])
            if not request_data:
                return
            
            # Execute code
            if request_data.get('type') == 'execute':
                result = self.execute_code(request_data.get('code', ''))
            else:
                result = {
                    'status': 'error',
                    'error': f"Unknown request type: {request_data.get('type')}"
                }
            
            # Write response
            self.write_response(command_id, result)
            
            # Mark as processed
            self.processed_requests.add(request_file['id'])
            
        except Exception as e:
            print(f'❌ Error processing request: {e}')
    
    def run(self):
        """Main processing loop"""
        print(f'\n🚀 Starting auto-processor')
        print(f'⏱️  Will run for {AUTO_RUN_DURATION//60} minutes')
        print(f'📁 Monitoring folder: {self.folder_id}')
        print(f'🔄 Poll interval: {POLL_INTERVAL}s\n')
        
        start_time = time.time()
        request_count = 0
        
        while time.time() - start_time < AUTO_RUN_DURATION:
            try:
                # Check for requests
                requests = self.list_requests()
                
                if requests:
                    print(f'\n📨 Found {len(requests)} new request(s)')
                    for req in requests:
                        self.process_request(req)
                        request_count += 1
                else:
                    elapsed = int(time.time() - start_time)
                    print(f'⏳ Waiting... ({elapsed}s elapsed, {request_count} processed)', end='\r')
                
                time.sleep(POLL_INTERVAL)
                
            except KeyboardInterrupt:
                print('\n\n🛑 Stopped by user')
                break
            except Exception as e:
                print(f'\n❌ Error in main loop: {e}')
                time.sleep(5)  # Wait before retrying
        
        print(f'\n\n✅ Auto-processor finished')
        print(f'📊 Processed {request_count} requests in {int(time.time()-start_time)}s')

# Auto-start the processor
print('🔄 Initializing processor...')
processor = AutoProcessor()

if processor.initialize():
    processor.run()
else:
    print('❌ Failed to initialize processor')
    print('Please ensure Drive is mounted and try again')
